# Part 2

### Recall that the HMM discussed in class is defined as follows:

<img src="images/hmm_eqn.jpg">

#### Data Preprocessing

In [1]:
import numpy as np
import pandas as pd

# Containers for the states and observations
tweets = []          # A list of all the tweets (Each tweet is a list)
word_count, tweet_count = 0, 0

# Import training data
with open('./SG/train', encoding='utf8') as f:
    training_lines = f.readlines()
    
    # For each line in the file
    for line in training_lines: 
        
        # If line is empty (i.e. we enter a new tweet)
        if line in['\n', '\r\n']: # Initialize a new tweet, reset word count
            if word_count != 0: #If the previous tweet was not empty, increase tweet count
                tweet_count += 1
            word_count = 0
        
        else:
            # Remove the spaces in each line
            stripped = line.strip().split(" ")
            if len(stripped) == 2:
                if word_count == 0:
                    tweets.append([tweet_count, word_count,'None','Start'])
                    word_count += 1
                tweets.append([tweet_count, word_count] + stripped)
                word_count += 1

#### Obtain count of labels

In [2]:
# Get Count(i) / Count(j)
df = pd.DataFrame(tweets,columns=['Tweet', 'Word', 'Observation', 'State'])
df = df.set_index(['Tweet', 'Word'])
states_counter = df.groupby('State').count().reset_index()
count_i_array = states_counter.as_matrix()
print(count_i_array)


[['B-negative' 1299]
 ['B-neutral' 5722]
 ['B-positive' 2613]
 ['I-negative' 443]
 ['I-neutral' 5272]
 ['I-positive' 1653]
 ['O' 91753]
 ['Start' 7094]]


In [3]:
transistion = df
transistion['J'] = transistion['State']
for Tweet, new_df in df.groupby(level=0):

    step = transistion.loc[Tweet,'State'].shift(-1).as_matrix()
    step.put(-1,'Stop')
    transistion.loc[Tweet,'J'] = step
transistion

Observation       State           J
Tweet Word                                                 
0     0                        None       Start           O
      1                         I'm           O           O
      2                          at           O   B-neutral
      3                    VivoCity   B-neutral           O
      4                          in           O   B-neutral
      5                   Singapore   B-neutral           O
      6     https://t.co/naeH7jxBhE           O        Stop
1     0                        None       Start           O
      1                           I           O           O
      2                        miss           O  B-positive
      3                    Stockton  B-positive           O
      4                        only           O           O
      5                         for           O           O
      6                         the           O           O
      7                        fact           O           O
      8                        that           O           O
      9                    wingstop           O           O
      10                        was           O           O
      11                       only           O           O
      12                          2           O           O
      13                    minutes           O           O
      14                       away           O           O
      15                       from           O           O
      16                         my           O           O
      17                      house           O        Stop
2     0                        None       Start   B-neutral
      1                        HSBC   B-neutral           O
      2                           :           O           O
      3                         RED           O           O
      4                       ALERT           O           O
...                             ...         ...         ...
7093  4                        left           O           O
      5                       lanes           O           O
      6                      closed           O           O
      7                           .           O           O
      8                          in           O           O
      9                        #I55           O           O
      10                         on           O           O
      11                          I           O           O
      12                         55           O           O
      13                         SB           O           O
      14                         at           O   B-neutral
      15                       4500   B-neutral   I-neutral
      16                          S   I-neutral   I-neutral
      17                          .   I-neutral   I-neutral
      18                   Broadway   I-neutral           O
      19                          ,           O           O
      20                       stop           O           O
      21                        and           O           O
      22                         go           O           O
      23                    traffic           O           O
      24                       back           O           O
      25                         to           O   B-neutral
      26                    Arsenal   B-neutral   I-neutral
      27                         St   I-neutral           O
      28                          ,           O           O
      29                      delay           O           O
      30                         of           O           O
      31                          5           O           O
      32                       mins           O           O
      33                #STLtraffic           O        Stop

[115849 rows x 3 columns]

In [4]:
count_transistion = transistion.groupby(['State','J']).count()
count_transistion['aij'] = count_transistion['Observation']
for i_state,i_sum in count_i_array:
    count_transistion.loc[i_state,'aij'] = count_transistion.loc[i_state,'aij'].as_matrix()/float(i_sum)

count_transistion

Observation       aij
State      J                                
B-negative B-negative            2  0.001540
           B-positive            1  0.000770
           I-negative          353  0.271747
           O                   913  0.702848
           Stop                 30  0.023095
B-neutral  B-negative            2  0.000350
           B-neutral            32  0.005592
           B-positive            4  0.000699
           I-neutral          2673  0.467144
           O                  2908  0.508214
           Stop                103  0.018001
B-positive B-neutral             6  0.002296
           B-positive            8  0.003062
           I-positive         1056  0.404133
           O                  1481  0.566781
           Stop                 62  0.023728
I-negative B-negative            2  0.004515
           B-neutral             1  0.002257
           I-negative           90  0.203160
           O                   342  0.772009
           Stop                  8  0.018059
I-neutral  B-neutral            17  0.003225
           I-neutral          2599  0.492982
           O                  2594  0.492033
           Stop                 62  0.011760
I-positive B-neutral             1  0.000605
           B-positive            5  0.003025
           I-positive          597  0.361162
           O                  1007  0.609195
           Stop                 43  0.026013
O          B-negative         1099  0.011978
           B-neutral          5262  0.057350
           B-positive         2293  0.024991
           O                 76313  0.831722
           Stop               6786  0.073959
Start      B-negative          194  0.027347
           B-neutral           403  0.056809
           B-positive          302  0.042571
           O                  6195  0.873273

### (5 pts) Write a function that estimates the emission parameters from the training set using MLE (maximum likelihood estimation):

<img src="images/MLE.jpg">

In [5]:
df["Count"] = 1
state_obs_counter = df.groupby(['State','Observation'],).count().reset_index()
count_y_o = state_obs_counter.as_matrix()

# Print State, Observation, Count
print(count_y_o)

[['B-negative' '#1MDB' 1 1]
 ['B-negative' '#AHS6' 1 1]
 ['B-negative' '#AustonMatthews' 2 2]
 ..., 
 ['O' '️' 226 226]
 ['O' '＠' 1 1]
 ['Start' 'None' 7094 7094]]


In [6]:
e = []
count_i_1d = count_i_array.flatten()
count_i_1d_list = count_i_1d.tolist()

for i in range(0, len(count_y_o)):
    y = count_y_o[i][0]
    index_y = count_i_1d_list.index(y)
    count_y = count_i_1d_list[index_y + 1]
    count_y_x = count_y_o[i][2]
    e_x_y = count_y_x / count_y
    e.append(e_x_y)
    
state_obs_counter["e(x|y)"] = e

In [7]:
print(state_obs_counter)

            State                      Observation     J  Count    e(x|y)
0      B-negative                            #1MDB     1      1  0.000770
1      B-negative                            #AHS6     1      1  0.000770
2      B-negative                  #AustonMatthews     2      2  0.001540
3      B-negative                          #Beatty     1      1  0.000770
4      B-negative                      #Blackhawks     1      1  0.000770
5      B-negative                             #C51     1      1  0.000770
6      B-negative                             #CNN     1      1  0.000770
7      B-negative                            #CSIS     1      1  0.000770
8      B-negative                           #China     1      1  0.000770
9      B-negative                         #Clinton     1      1  0.000770
10     B-negative                  #CrookedHillary     1      1  0.000770
11     B-negative                       #DailyMail     1      1  0.000770
12     B-negative                    #

### (10 pts) During the testing phase, if the word does not appear in the “modified training set”, we replace that word with #UNK# as well. Set k to 3, implement this fix into your function for computing the emission parameters.

In [8]:
k = 3

# Replace observation words with #UNK# for observations with count <= 3
state_obs_counter['Observation'].loc[state_obs_counter['Count'] < k] = '#UNK#'

print(state_obs_counter)

            State      Observation     J  Count    e(x|y)
0      B-negative            #UNK#     1      1  0.000770
1      B-negative            #UNK#     1      1  0.000770
2      B-negative            #UNK#     2      2  0.001540
3      B-negative            #UNK#     1      1  0.000770
4      B-negative            #UNK#     1      1  0.000770
5      B-negative            #UNK#     1      1  0.000770
6      B-negative            #UNK#     1      1  0.000770
7      B-negative            #UNK#     1      1  0.000770
8      B-negative            #UNK#     1      1  0.000770
9      B-negative            #UNK#     1      1  0.000770
10     B-negative            #UNK#     1      1  0.000770
11     B-negative            #UNK#     1      1  0.000770
12     B-negative            #UNK#     1      1  0.000770
13     B-negative            #UNK#     1      1  0.000770
14     B-negative            #UNK#     1      1  0.000770
15     B-negative            #UNK#     1      1  0.000770
16     B-negat

C:\Users\Pinar\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


### (10 pts) Implement a simple sentiment analysis system that produces the tag y* = arg max e(x|y) for each word x in the sequence